In [1]:
import sqlite3
import pandas as pd 
import sys
from IPython.display import display, HTML
sys.path.insert(0, '../../edit-pathways/util/')
sys.path.insert(0, '../scripts/')
import util_newssniffer_parsing as unp 
import spacy
import neuralcoref
# nlp = spacy.load('en_core_web_sm')

In [2]:
nlp = spacy.load('en_core_web_lg')

# Try seeing if neuralcoref will help

In [546]:
# Let's try before using the conversion dictionary:
try:
    nlp.remove_pipe('neuralcoref')
except:
    pass

neuralcoref.add_to_pipe(nlp, max_dist=500, conv_dict={'Deepika': ['entity']})
doc = nlp(u'Deepika has a dog. She loves him. The movie star has always been fond of animals')
doc._.coref_resolved

'Deepika has a dog. The movie star loves a dog. The movie star has always been fond of animals'

In [3]:
doc._.coref_clusters

[Deepika: [Deepika, She, him, The movie star]]

In [7]:
db_path = '../../edit-pathways/data/diffengine-diffs/spark-output/nyt-matched-sentences.db'
db_path = '../../edit-pathways/data/diffengine-diffs/db/newssniffer-nytimes.db'
conn = sqlite3.connect(db_path)
df = pd.read_sql('SELECT * from entryversion limit 5', con=conn)
t = df['summary'][0].replace('</p><p>', ' ')

In [9]:
doc = nlp(t)

In [ ]:
doc._.coref_clusters

In [60]:
s_pre = df['summary'][0].split('</p><p>')
s_fixed = [] 
for s in s_pre:
    doc = nlp(s)
    s_fixed.append(doc._.coref_resolved)

In [61]:
htmls = []
for s_o, s_n in zip(s_pre, s_fixed):
    v_o, v_n = unp.html_compare_sentences(s_o, s_n)
    htmls.append((v_o, v_n))

In [65]:
all_html = [] # '<table>%s</table>'
for v_o, v_n in htmls[:5]:
    all_html.append('<tr><td>%s</td><td>%s</td></tr>' % (v_o, v_n))
display(HTML('<table>%s</table>' % ''.join(all_html)))

"In Silicon Valley , Apple just won big against Samsung in the patent lawsuit of the year , after trading claims and counterclaims of pilfered product ideas . Across the country , in a federal court in Florida , an inventor named Mark Stadnyk is waging a different kind of patent warfare — an ambitious and perhaps quixotic legal foray .","In Silicon Valley , Apple just won big against Samsung in the patent lawsuit of the year , after trading claims and counterclaims of pilfered product ideas . Across the country , in a federal court in Florida , an inventor named Mark Stadnyk is waging a different kind of patent warfare — an ambitious and perhaps quixotic legal foray ."
"Mr. Stadnyk , who holds a patent on a motorcycle windshield , is suing the United States government , aiming to head off a patent law that he says will favor big companies and hurt lone inventors like himself .","Mr. Stadnyk , who holds a patent on a motorcycle windshield , is suing the United States government , aiming to head off a patent law that he says will favor big companies and hurt lone inventors like he ."
"Represented by a prominent Washington lawyer , Mr. Stadnyk filed a suit last month that challenges the constitutionality of legislation that Congress passed last fall , the America Invents Act . Mr. Stadnyk and his lawyer — along with some academics , entrepreneurs and venture capitalists — assert that the legislation is a triumph of corporate lobbying power over the founders ’ wishes , and that it threatens America ’s stature as the world ’s leading innovator .","Represented by a prominent Washington lawyer , Mr. Stadnyk filed a suit last month that challenges the constitutionality of legislation that Congress passed last fall , the America Invents Act . Mr. Stadnyk and Mr. Stadnyk lawyer — along with some academics , entrepreneurs and venture capitalists — assert that a suit last month that challenges the constitutionality of legislation that Congress passed last fall , the America Invents Act is a triumph of corporate lobbying power over the founders ’ wishes , and that a suit last month that challenges the constitutionality of legislation that Congress passed last fall , the America Invents Act threatens America ’s stature as the world ’s leading innovator ."
"The present system , one of the nation ’s oldest patent principles and called “ first to invent , ” relies on lab notebooks , e - mails and early prototypes to establish the date of invention . The impending law would overturn that by awarding patents to the inventors who are “ first to file ” with the United States Patent and Trademark Office .","The present system , one of the nation ’s oldest patent principles and called “ first to invent , ” relies on lab notebooks , e - mails and early prototypes to establish the date of invention . The impending law would overturn that by awarding patents to the inventors who are “ first to file ” with the United States Patent and Trademark Office ."
"Mr. Stadnyk , 48 , a garage inventor who stumbled into the world of patents after he bought a powerful new motorcycle and wanted to avoid being battered by the wind when riding at 60 miles an hour , even with a windshield . He devised a system of brackets and gears to adjust the height and angle of the windshield and the gap between it and the motorcycle . With his system , he says , the rider feels a flutter of breeze instead of jolting winds and turbulence .","Mr. Stadnyk , 48 , a garage inventor who stumbled into the world of patents after he bought a powerful new motorcycle and wanted to avoid being battered by the wind when riding at 60 miles an hour , even with a windshield . he devised a system of brackets and gears to adjust the height and angle of a windshield and the gap between it and a powerful new motorcycle . With he system , he says , the rider feels a flutter of breeze instead of jolting winds and turbulence ."


# AllenNLP Coref

In [ ]:
from allennlp.predictors.predictor import Predictor

model_url = 'https://storage.googleapis.com/allennlp-public-models/coref-spanbert-large-2020.02.27.tar.gz'
predictor = Predictor.from_path(model_url)  # load the model

In [547]:
text = "Eva and Martha didn't want their friend Jenny to feel lonely so they invited her to the party."
prediction = predictor.predict(document=text)  # get the prediction

print(prediction['clusters'])  # list of clusters (the indices of spaCy tokens)
# [[[0, 2], [6, 6], [13, 13]], [[6, 8], [15, 15]]]

print(predictor.coref_resolved(text))  # resolved text

[[[0, 2], [6, 6], [13, 13]], [[6, 8], [15, 15]]]
Eva and Martha didn't want Eva and Martha's friend Jenny to feel lonely so Eva and Martha invited their friend Jenny to the party.


In [119]:
pred = predictor.predict(t)

In [122]:
t_r = predictor.coref_resolved(t)

In [125]:
q = util.get_quotes_method_2(text=t_r, cluster=True, resolve_coref=False, greedyness=.4, max_dist=150)

In [721]:
# clusters = predictor.predict(t)['clusters']
doc = nlp(t)

In [1510]:
fuzzy = FuzzyIntersectionStrategy(predictor, nlp)
c = fuzzy.clusters(t)

## extract quotes using replaced document

In [521]:
t_int = improved_replace_corefs(doc, c)
t_r_imp = improved_replace_corefs(doc, clusters)
q1 = util.get_quotes_method_2(text=t_int, cluster=False, resolve_coref=False, greedyness=.4, max_dist=150)

In [527]:
doc = nlp(t_int)

In [528]:
for s in doc.sents:
    print(s)
    print()

In Silicon Valley, Apple just won big against Samsung in the patent lawsuit of the year, after trading claims and counterclaims of pilfered product ideas.

Across the country, in a federal court in Florida, an inventor named Mark Stadnyk is waging a different kind of patent warfare — an ambitious and perhaps quixotic legal foray.

Mr. Stadnyk, who holds a patent on a motorcycle windshield, is suing the United States government, aiming to head off a patent law that the legislation says will favor big companies and hurt lone inventors like an inventor named Mark Stadnyk.

Represented by a prominent Washington lawyer, an inventor named Mark Stadnyk filed a suit last month that challenges the constitutionality of legislation that Congress passed last fall, the legislation.

Mr. Stadnyk and an inventor named Mark Stadnyk's lawyer — along with some academics, entrepreneurs and venture capitalists

— assert that the legislation is a triumph of corporate lobbying power over the founders’ wishe

# Use coref-clusters to merge extracted sources

In [555]:
doc = nlp(t)

# Try backtracking from Named Entities (PERSON) first

In [3]:
import coref_resolution_util

In [11]:
from importlib import reload
reload(coref_resolution_util)

<module 'coref_resolution_util' from '../scripts/coref_resolution_util.py'>

In [ ]:
import sqlite3
import pandas as pd 
import sys
from IPython.display import display, HTML
sys.path.insert(0, '../../edit-pathways/util/')
sys.path.insert(0, '../scripts/')
import util_newssniffer_parsing as unp 
import spacy
import neuralcoref

conn = sqlite3.connect(db_path)
df = pd.read_sql('SELECT * from entryversion limit 5', con=conn)
t = df['summary'][0].replace('</p><p>', ' ')

In [14]:
quote_idxes, sent_words, sent_lens = coref_resolution_util.perform_quote_extraction_and_clustering(t)

In [1508]:
back_span_col = 'Crimson'
back_sent_col = 'DarkSeaGreen'
quote_sent_col = 'Gold'
quote_span_col = 'HotPink'
head_span_col = 'Lime'

t2 = t.iloc[13]

q_sents = set(map(lambda x: x[0], t2['quote_chunks']))
b_sents = set(map(lambda x: x[0], t2['background_chunks']))

all_sents_html = []
for s_idx, sent in enumerate(sent_words):
    adj = sum(sent_lens[:s_idx])
    if s_idx == t2['head_s_idx']:
        sent, adj = format_span_with_word_list(adj, sent, t2['head_span'], head_span_col)
    # format sentence
    if s_idx in q_sents:
        q_chunks = list(filter(lambda x: x[0] == s_idx, t2['quote_chunks']))
        for _, q_span in q_chunks:
            sent, adj = format_span_with_word_list(adj, sent, q_span, quote_span_col)
        sent = format_sent_with_word_list(sent, quote_sent_col)
    elif s_idx in b_sents:
        b_chunks = list(filter(lambda x: x[0] == s_idx, t2['background_chunks']))
        for _, b_span in b_chunks:
            sent, adj = format_span_with_word_list(adj, sent, b_span, back_span_col)
        sent = format_sent_with_word_list(sent, back_sent_col)
    else:
        sent = ' '.join(sent)
    all_sents_html.append(sent)

display(HTML('<br><br>'.join(all_sents_html).replace('$', '\$')))

In [1484]:
doc[t2['head_span'][0]:t2['head_span'][1]]

Barbara Boxer

In [ ]:
doc[t2['head_span'][0]:t2['head_span'][1]]

# Checks

In [1211]:
list(doc.sents)[74]

In Silicon Valley, Stephen G. Perlman, a high-tech inventor and entrepreneur, worries about the law’s practical effect.

In [1218]:
q = extract_quotes_from_nsubj(doc, return_dict=True)

In [ ]:
for possible_subject in list(doc.sents)[74]:
#     if possible_subject.text == 'Perlman':
#         break
#     print(possible_subject)
#     print(possible_subject.pos_)
#     print()
    if (
        possible_subject.dep_ == 'nsubj' and 
        possible_subject.head.pos_ == 'VERB'
    ):
        print(possible_subject)
    
#     if False:
#         if (
#             possible_subject.dep_ == 'nsubj' and 
#             possible_subject.head.pos_ == 'VERB' and 
#             possible_subject.head.text in (
#                 'say', 'says', 'said', 
#                 'describe', 'describes', 'described', 
#                 'claims', 'claims', 'claimed', 
#                 'explained', 'explains', 'explain',
#                 'mentioned', 'mentions', 'mention',
#                 'articulated', 'articulates', 'articulate',
#                 'called', 'calls', 'call',
#                 'declared', 'declares', 'declare',
#                 'worried', 'worries', 'worry',
#            )
#         ):
#             print(possible_subject)

In [1228]:
possible_subject.head

worries

In [1230]:
possible_subject.head.pos_

'NOUN'

In [560]:
cluster_mapper = defaultdict(list)
seen = set()
for k in entities.keys():
    if k in span_to_head_mapper:
        referent_span = span_to_head_mapper[k]
        cluster_mapper[(referent_span[0], referent_span[1] + 1)].append(k)
        seen.add(k)

for k in entities.keys():
    if k not in seen:
        cluster_mapper[k].append(k)

In [561]:
from importlib import reload
import parsing_util

In [562]:
reload(parsing_util)

<module 'parsing_util' from '../scripts/parsing_util.py'>

In [563]:
def merge_clusters(entities, clusters=None, cluster_mapper=None):
    if cluster_mapper == None:
        cluster_mapper = {}
        for cluster in clusters:
            key = max(cluster, key=lambda x: len(x))
            cluster_mapper[key] = cluster

    # group for output
    entities_clustered = defaultdict(lambda: {'background sentence': [], 'quote sentence': [], 'span_text': []})
    for c_key, cluster in cluster_mapper.items():
        for c_i in cluster:
            for part in ['background sentence', 'quote sentence', 'span_text']:
                to_add = entities[c_i][part]
                if isinstance(to_add, list):
                    entities_clustered[c_key][part].extend(to_add)
                else:
                    entities_clustered[c_key][part].append(to_add)
    
    ## 
    return entities_clustered

In [564]:
entities = merge_clusters(entities, cluster_mapper=cluster_mapper)

In [565]:
def convert_entities_dict_to_text(entities, doc):
    output_entities = {}
    for k, v in entities.items():
        key_text = doc[k[0]: k[1]].text
        q_sent, ms = [], []
        for q in v['quote sentence']:
            q_sent.append((q[0], q[1].text))
        for m in v['span_text']:
            ms.append(m.text)
        output_entities[key_text] = {'quote sentence': q_sent, 'entity mentions': ms, 'background sentence': []}
    return output_entities 

In [566]:
convert_entities_dict_to_text(entities, doc)

{'the America Invents Act': {'quote sentence': [(2,
    'Mr. Stadnyk, who holds a patent on a motorcycle windshield, is suing the United States government, aiming to head off a patent law that he says will favor big companies and hurt lone inventors like himself.')],
  'entity mentions': ['he'],
  'background sentence': []},
 'an inventor named Mark Stadnyk': {'quote sentence': [(10,
    'With his system, he says, the rider feels a flutter of breeze instead of jolting winds and turbulence.'),
   (14, 'Yearly sales, he said, are more than $500,000 and growing briskly.'),
   (22,
    'He says he studied the proposals and the law, read blogs and reached out to Washington lawyers and academics who raised the issue of its constitutionality.'),
   (40,
    'My property would be given away by the government under this new law,” he said.')],
  'entity mentions': ['he', 'he', 'He', 'he'],
  'background sentence': []},
 'Senator Barbara Boxer, Democrat of California': {'quote sentence': [(49,
  

In [567]:
entities

defaultdict(<function __main__.merge_clusters.<locals>.<lambda>()>,
            {(127, 131): {'background sentence': [],
              'quote sentence': [(2,
                Mr. Stadnyk, who holds a patent on a motorcycle windshield, is suing the United States government, aiming to head off a patent law that he says will favor big companies and hurt lone inventors like himself.)],
              'span_text': [he]},
             (39, 44): {'background sentence': [],
              'quote sentence': [(10,
                With his system, he says, the rider feels a flutter of breeze instead of jolting winds and turbulence.),
               (14,
                Yearly sales, he said, are more than $500,000 and growing briskly.),
               (22,
                He says he studied the proposals and the law, read blogs and reached out to Washington lawyers and academics who raised the issue of its constitutionality.),
               (40,
                My property would be given away by th

# Extract quotes from some documents

In [2]:
sys.path.insert(0, '../scripts/')
sys.path.insert(0, '../')

In [3]:
import util

In [11]:
t2 = t.replace('</p><p>', ' ')

In [17]:
quotes = util.get_quotes_method_2(text=t2, cluster=True, resolve_coref=True, greedyness=.4, max_dist=150)

In [18]:
quotes

defaultdict(<function util.cluster_entities_method_1.<locals>.<lambda>()>,
            {'he': {'background sentence': [],
              'quote sentence': [(2,
                'Mr. Stadnyk , who holds a patent on a motorcycle windshield , is suing the United States government , aiming to head off a patent law that he says will favor big companies and hurt lone inventors like himself .'),
               (2,
                'Mr. Stadnyk , who holds a patent on a motorcycle windshield , is suing the United States government , aiming to head off a patent law that he says will favor big companies and hurt lone inventors like himself .'),
               (9,
                'With he system , he says , the rider feels a flutter of breeze instead of jolting winds and turbulence .'),
               (9,
                'With he system , he says , the rider feels a flutter of breeze instead of jolting winds and turbulence .'),
               (9,
                'With he system , he says , the rider

In [19]:
quotes.keys()

dict_keys(['he', 'the United States government', 'Mark Stadnyk', 'opponents', 'Large corporations', 'who', 'Senator Barbara Boxer', 'Jonathan S. Massey', 'The new patent law', 'suit', 'some legal experts', 'courts', 'Arti K. Rai', 'Mark A. Lemley', 'they', 'Stephen G. Perlman', 'Gary Lauder', 'Patty', 'Obama', 'William J. Brennan Jr.', 'Laurence H. Tribe', 'Al Gore', 'Bush'])

In [20]:
quotes['Mark Stadnyk']

{'background sentence': [(3, '”'),
  (4, '”'),
  (7, '”'),
  (10, '”'),
  (11, '”'),
  (14, '”'),
  (14, '”'),
  (20, '”'),
  (22, '”'),
  (32, '”'),
  (32, '”'),
  (43, '”'),
  (1, '”')],
 'quote sentence': [(21,
   'Mr. Stadnyk says Mr. Stadnyk studied the proposals and the law , read blogs and reached out to Washington lawyers and academics who raised the issue of its constitutionality .'),
  (34,
   'Others took on Obamacare , and this is my fight , ” said Mr. Stadnyk , who lives in Ridge Manor ,'),
  (21,
   'Mr. Stadnyk says Mr. Stadnyk studied the proposals and the law , read blogs and reached out to Washington lawyers and academics who raised the issue of its constitutionality .'),
  (21,
   'Mr. Stadnyk says Mr. Stadnyk studied the proposals and the law , read blogs and reached out to Washington lawyers and academics who raised the issue of its constitutionality .'),
  (34,
   'Others took on Obamacare , and this is my fight , ” said Mr. Stadnyk , who lives in Ridge Manor ,')]

In [27]:
deduped_output

{'he': {'background sentence': [],
  'quote sentence': [(2,
    'Mr. Stadnyk , who holds a patent on a motorcycle windshield , is suing the United States government , aiming to head off a patent law that he says will favor big companies and hurt lone inventors like himself .'),
   (9,
    'With he system , he says , the rider feels a flutter of breeze instead of jolting winds and turbulence .'),
   (13,
    'Yearly sales , he said , are more than $ 500,000 and growing briskly .'),
   (40,
    'My property would be given away by the government under this new law , ” he said .'),
   (58,
    'The new patent law , he said , “ violates the text and the original understanding of the Constitution .'),
   (73, 'and they may be right in not liking the outcome , ” he said .'),
   (80,
    'The new law , he said , with its emphasis on speed , could well make it more difficult to wait before filing an application , to take the time to experiment over years , and to confidentially show early conce

In [38]:
doc = nlp(u'My sister has a dog. She loves him')

doc._.coref_clusters
doc._.coref_clusters[1].mentions
doc._.coref_clusters[1].mentions[-1]
doc._.coref_clusters[1].mentions[-1]._.coref_cluster.main

a dog

In [39]:
doc._.coref_clusters[1].mentions[-1]

him

In [35]:
doc._.coref_clusters[1].mentions[-1] = 'her'

In [40]:
doc._.coref_clusters[1].mentions

[a dog, him]

In [41]:
doc._.coref_scores

{My sister: {My sister: 1.3110305070877075},
 a dog: {a dog: 1.804752230644226, My sister: -1.6715972423553467},
 She: {She: 0.09481996297836304,
  My sister: 7.786711692810059,
  a dog: -1.1784329414367676},
 him: {him: -1.4293134212493896,
  My sister: 2.0916330814361572,
  a dog: 3.683906316757202,
  She: -3.400832176208496}}

In [54]:
doc._.coref_resolved

'My sister has a dog. My sister loves a dog'

In [43]:
long_doc = nlp(t2)

In [96]:
s = long_doc._.coref_scores

In [98]:
long_doc._.coref_clusters

[Mr. Stadnyk: [an inventor named Mark Stadnyk, he, himself, Mr. Stadnyk, his],
 a suit last month that challenges the constitutionality of legislation that Congress passed last fall, the America Invents Act: [a suit last month that challenges the constitutionality of legislation that Congress passed last fall, the America Invents Act, the legislation, it],
 he: [he, He, his, he, his, his, he, himself, His, he, he],
 a powerful new motorcycle: [a powerful new motorcycle, the motorcycle],
 a windshield: [a windshield, the windshield],
 MadStad: [MadStad, his],
 the United States: [the United States, the United States],
 Mr. Stadnyk: [Mr. Stadnyk, he, Mr. Stadnyk, He, he, its, he],
 the patent legislation: [the patent legislation, it],
 the law: [the law, the new law, the new law, this new law, The new patent law, The new patent law],
 Washington: [Washington, Washington],
 Japan and elsewhere: [Japan and elsewhere, them],
 America: [America, America],
 President Obama: [President Obama, 

In [60]:
doc

My sister has a dog. She loves him

In [75]:
s = doc._.coref_clusters[0].mentions[1]

In [76]:
s.start

6

In [77]:
s.end

7

In [94]:
doc[s.start:s.end]

She

In [33]:
import unidecode
doc = nlp(unidecode.unidecode(t2))

In [652]:
article_batch = pd.read_sql('SELECT * from entryversion where version = 0 limit 500', con=conn)

In [657]:
from tqdm.auto import tqdm

In [ ]:
all_named_ents = []
for a in tqdm(article_batch['summary']):
    doc = nlp(a)
    person_ents = list(set(filter(lambda x: x.label_ == 'PERSON', doc.ents)))
    all_named_ents.extend(person_ents)

In [665]:
names = list(set(map(lambda x: x.text, all_named_ents)))